## INIZIALIZATION

In [ ]:
import numpy as np
import time


problems = [{}]

solutions = np.empty(3, dtype=object)

MAX_STEPS=1000

# Random initialization constants  
DELAY=1 # every DELAY seconds (N_ITEMS / STEPS) are set unassigned
STEPS=100

SIMULATED_ANNEALING_TEMP=10000  # 100 - 20000

# with this configuration it should take around 5 minutes to solve all the problems

# KNAPSACKS PROBLEMS

### Problem 1:

In [ ]:
rng = np.random.default_rng(seed=42)
problems[0] = {
    "NUM_KNAPSACKS": 3,
    "NUM_ITEMS": 20,
    "NUM_DIMENSIONS": 2
}
problems[0]["VALUES"] = rng.integers(0, 100, size=problems[0]["NUM_ITEMS"])
problems[0]["WEIGHTS"] = rng.integers(0, 100, size=(problems[0]["NUM_ITEMS"], problems[0]["NUM_DIMENSIONS"]))
problems[0]["CONSTRAINTS"] = rng.integers(0, 100 * problems[0]["NUM_ITEMS"] // problems[0]["NUM_KNAPSACKS"], size=(problems[0]["NUM_KNAPSACKS"], problems[0]["NUM_DIMENSIONS"]))


### Problem 2:

In [ ]:
rng = np.random.default_rng(seed=42)
problems.append({})
problems[1] = {
    "NUM_KNAPSACKS": 10,
    "NUM_ITEMS": 100,
    "NUM_DIMENSIONS": 10
}
problems[1]["VALUES"] = rng.integers(0, 1000, size=problems[1]["NUM_ITEMS"])
problems[1]["WEIGHTS"] = rng.integers(0, 1000, size=(problems[1]["NUM_ITEMS"], problems[1]["NUM_DIMENSIONS"]))
problems[1]["CONSTRAINTS"] = rng.integers(1000 * 2, 1000 * problems[1]["NUM_ITEMS"] // problems[1]["NUM_KNAPSACKS"], size=(problems[1]["NUM_KNAPSACKS"], problems[1]["NUM_DIMENSIONS"]))

### Problem 3:

In [ ]:
rng = np.random.default_rng(seed=42)
problems.append({})
problems[2] = {
    "NUM_KNAPSACKS": 100,
    "NUM_ITEMS": 5000,
    "NUM_DIMENSIONS": 100
}
problems[2]["VALUES"] = rng.integers(0, 1000, size=problems[2]["NUM_ITEMS"])
problems[2]["WEIGHTS"] = rng.integers(0, 1000, size=(problems[2]["NUM_ITEMS"], problems[2]["NUM_DIMENSIONS"]))
problems[2]["CONSTRAINTS"] = rng.integers(1000 * 10, 1000 * 2 * problems[2]["NUM_ITEMS"] // problems[2]["NUM_KNAPSACKS"], size=(problems[2]["NUM_KNAPSACKS"], problems[2]["NUM_DIMENSIONS"]))

## Elaboration:

In [ ]:
def solve_knapsack(problem): #i treat all knapsacks together

    main_rng = np.random.default_rng(seed=int(time.time() * 1000000000))

    def check_constraints(solution):
        total_weights = np.zeros((problem["NUM_KNAPSACKS"], problem["NUM_DIMENSIONS"]), dtype=int)
        
        # Iterate through items and accumulate weights for assigned knapsacks
        for i, knapsack_id in enumerate(solution):
            if knapsack_id >= 0:  # -1 means unassigned, 0+ means knapsack index 
                for d in range(problem["NUM_DIMENSIONS"]):
                    total_weights[knapsack_id, d] += problem["WEIGHTS"][i, d]
        
        return np.all(total_weights <= problem["CONSTRAINTS"])

    # Create random assignment: -1 = unassigned, 0,1,2,... = knapsack index
    def random_starting_solution(n_knapsack, n_items):
        forced_unassigned = set()  # Set to track positions to force to -1 (unassigned)
        last_time = time.time()
        
        while True:
            solution = main_rng.integers(-1, n_knapsack, size=n_items)
            
            for pos in forced_unassigned:
                solution[pos] = -1

            #it will exit at least with the solution without assignements
            if check_constraints(solution):     
                break
            
            current_time = time.time()
            if current_time - last_time >= DELAY: 
                random_positions = main_rng.integers(0, n_items, size=n_items // STEPS)
                forced_unassigned.update(random_positions)  
                last_time = current_time
                
        return solution

    def vector_to_matrix(item_assignments):
        matrix = np.zeros((problem["NUM_KNAPSACKS"], problem["NUM_ITEMS"]), dtype=int)
        for i, knapsack_id in enumerate(item_assignments):
            if knapsack_id >= 0:  
                matrix[knapsack_id, i] = 1  
        return matrix

    def tweak(solution):
        while True:
            new_solution = solution.copy()
            i = main_rng.integers(0, problem["NUM_ITEMS"])
            new_solution[i] = main_rng.integers(-1, problem["NUM_KNAPSACKS"])
            
            if check_constraints(new_solution):
                return new_solution

    def fitness(solution):
        total_value = 0
        for i, knapsack_id in enumerate(solution):
            if knapsack_id >= 0:  
                total_value += problem["VALUES"][i]
        return total_value

    solution = random_starting_solution(problem["NUM_KNAPSACKS"], problem["NUM_ITEMS"])
    
    step = 0
    while step < MAX_STEPS:
        step += 1
        new_solution = tweak(solution)  # Always returns valid solution
        if fitness(new_solution) >= fitness(solution):
            solution = new_solution
            #print(fitness(solution))
        else:
            acceptance_prob = np.exp((fitness(solution) - fitness(new_solution)) / SIMULATED_ANNEALING_TEMP)
            if main_rng.random() < acceptance_prob:
                solution = new_solution
                #print("worse solution accepted")
                #print(fitness(solution))

    # Convert back to 2D matrix for return (to match expected output format)
    return vector_to_matrix(solution)

In [ ]:
i=0
for problem in problems :
    #print(problem)
    #print (i)
    solutions[i] = solve_knapsack(problem)
    #print(solutions[i] )
    i += 1


In [ ]:
# check the last problem solution is not empty
for k in range(len(solutions[2][1])):
    if solutions[2][1][k]:
        print (k)